In [ ]:
!git clone https://github.com/cye2020/TSF-SPI.git

In [ ]:
cd TSF-SPI/

In [ ]:
!git pull

In [ ]:
# User-defined Data preprocessor
from utils.data_loader import DataLoader
from utils.spliter import WindowGenerator

# User-defined Visualization and Print Results
from utils.plot_util import plot
from utils.evaluation import printResult

# Model Callbacks
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard

# Model Optimizer
from tensorflow.keras.optimizers import Adam, RMSprop, SGD

# Dataframe
import pandas as pd
import numpy as np

# Model Structure
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv1D, MaxPool1D, Input, Bidirectional, LSTM
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, InputLayer
from tensorflow.keras import backend as K

# Seed
from tensorflow.random import set_seed
from numpy.random import randint

In [ ]:
data_loader = DataLoader()
Target = 'Kospi'

features = [Target, 'USD/KRW', 'GDP', 'IAIP', 'LIR', 'M1', 'disease']

train, test, date_lists = data_loader.load_csv(path='./data/kospi.csv', 
                                               features=np.concatenate((['Date'],features), axis=0), 
                                               split_date='2021-01-01')


In [ ]:
data = pd.read_csv('./data/kospi.csv')
data.isnull().any()

In [ ]:
boolarr = np.isnan(test)
boolarr.sum()
np.count_nonzero(boolarr)

In [ ]:
seedNum = 3394
set_seed(seedNum)
np.random.seed(seedNum)

In [ ]:
CNN_input = (30, 1, 1)

wg_1 = WindowGenerator(train, input_width=CNN_input[0], label_width=CNN_input[1], shift=CNN_input[2])
twg_1 = WindowGenerator(test, input_width=CNN_input[0], label_width=CNN_input[1], shift=CNN_input[2])

train_cnn_x, train_cnn_y = wg_1.split_window()
test_cnn_x, test_cnn_y = twg_1.split_window()

In [ ]:
model = Sequential()
model.add(Conv1D(64, 2, input_shape=(30,7)))
model.add(Activation('relu'))
model.add(MaxPool1D(pool_size = 2))

model.add(Flatten())
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(1))

In [ ]:
model.summary()

In [ ]:
# 모델의 학습률입니다.
Lr = 0.001

# 옵티마이저 함수입니다.
Optimizer = Adam(learning_rate=Lr)

# 학습하는 최대 Epoch(iterations)입니다.
Epochs = 100

# Train 데이터 중 validation 데이터의 비율을 정합니다. 
Validation_split=0.2

# Batch size입니다.
Batch_size = 64

# 손실함수입니다.
Loss = 'mean_squared_error'

#callbacks
es = EarlyStopping(monitor='val_loss', min_delta=1e-10, patience=10, verbose=1)
rlr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1)

In [ ]:
model.compile(optimizer=Optimizer, loss=Loss)

history = model.fit(train_cnn_x,
                    train_cnn_y,
                    shuffle=True,
                    epochs=Epochs,
                    callbacks=[es, rlr],
                    validation_split=Validation_split,
                    verbose=1,
                    batch_size=Batch_size)

In [ ]:
train_cnn_predict = model.predict(train_cnn_x)
test_cnn_predict = model.predict(test_cnn_x)

In [ ]:
TRUE = pd.DataFrame(data_loader.original[:, 0:1], columns=["True"]).set_index(pd.Series(date_lists[0]))
prediction_train = wg_1.to_pandas(train_cnn_predict[1:][:,-1], date_lists[1], name="Train")
prediction_test = twg_1.to_pandas(test_cnn_predict[1:][:,-1], date_lists[2], name="Test")

plot(TRUE, prediction_train, prediction_test, show=True)

In [ ]:
printResult(date_lists[2], test_cnn_y, test_cnn_predict, Target, features)